# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [2]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [3]:
questions, answers = get_questions_and_answers(faq_path="../data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [52]:
from sentence_transformers import SentenceTransformer, util
from spellchecker import SpellChecker
import spacy


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = get_sentence_embeddings(model, dataframe.Question)
    message_embed = model.encode(message)

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index


def remove_stop_words(text:str):
    nlp_model = spacy.load("pt_core_news_sm")
    text_tokens = nlp_model(text)

    words_without_stopwords = []

    for token in text_tokens:
        if not token.is_stop:
            words_without_stopwords.append(token.text)
    
    text_without_stopwords = " ".join(words_without_stopwords)

    return text_without_stopwords


def get_sentence_embeddings(model, series):
    """
        This function removes all 'stop words' for each sentence in a pandas.Series
        and then creates a embed for the sentence without the stop words.
    """

    text_embeddings = []

    for text in series:
        embed = model.encode(remove_stop_words(text))
        text_embeddings.append(embed)
    
    return pd.Series(text_embeddings)

In [42]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [43]:
message = "é possível melhorar meu plano?"
message = remove_stop_words(message)

### 2.1.1 all-mpnet-base-v2 model

In [44]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."


### 2.1.2 multi-qa-mpnet-base-dot-v1

In [45]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
26,A empresa oferece serviços de telefonia VoIP?,"Sim, oferecemos serviços de telefonia VoIP par..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
14,A empresa oferece roteadores e equipamentos pa...,"Sim, fornecemos equipamentos para internet em ..."


### 2.1.3 all-distilroberta-v1 model

In [46]:
model = SentenceTransformer("all-distilroberta-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."


### 2.1.4 all-MiniLM-L6-v2 model

In [47]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...


### 2.1.5 multi-qa-MiniLM-L6-cos-v1

In [48]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
11,A empresa oferece descontos para pacotes combi...,"Sim, oferecemos descontos especiais para clien..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...


### 2.1.6 msmarco-distilbert-base-v4

In [49]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

melhorar plano ?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
17,Como posso solicitar a mudança de endereço dos...,Entre em contato com nosso atendimento para so...
23,A empresa oferece serviço de fibra óptica?,"Sim, nossa empresa oferece serviço de internet..."
12,O que fazer em caso de falha no serviço de int...,"Em caso de falha no serviço, entre em contato ..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
